### Libraries

In [1]:
from binance.client import Client
import pandas as pd
import ta
import datetime
import time

### Credentials

In [2]:
api_key = "binance_api_key"
api_secret = "binance_secret_key"

### Pulling Crypto Price Data

In [3]:
client = Client(api_key,api_secret, testnet=True)

In [4]:
symbol = 'BTCUSDT'

In [5]:
pd.DataFrame(client.get_historical_klines('BTCUSDT' , '15m', '3000 minutes UTC'))

,0,1,2,3,4,5,6,7,8,9,10,11
0,1684423800000,27119.66000000,50000.00000000,9960.10000000,27139.33000000,93.54912300,1684424699999,2539568.80403616,3017,52.31112200,1421464.36251065,0
1,1684424700000,27139.07000000,29000.00000000,13569.90000000,27086.56000000,110.85593600,1684425599999,3003959.34413831,2459,56.42707600,1530410.42844863,0
2,1684425600000,27086.69000000,29024.90000000,25000.00000000,27057.94000000,113.76933100,1684426499999,3080811.96534037,2816,64.01864400,1733924.82760658,0
3,1684426500000,27057.32000000,29000.00000000,24910.60000000,27031.55000000,90.40236300,1684427399999,2445212.37491389,2094,49.13869100,1329271.90425130,0
4,1684427400000,27033.53000000,27142.00000000,26758.00000000,27102.52000000,80.49941400,1684428299999,2178640.81494047,1955,47.45263600,1284297.30537925,0
...,...,...,...,...,...,...,...,...,...,...,...,...
195,1684599300000,26941.04000000,26941.04000000,26920.63000000,26931.07000000,27.94597900,1684600199999,752599.32040207,842,12.86653200,346499.07347337,0
196,1684600200000,26931.08000000,26932.00000000,26918.83000000,26923.15000000,21.73213400,1684601099999,585124.10299767,644,11.00720400,296355.90188705,0
197,1684601100000,26923.15000000,26923.16000000,26913.22000000,26918.46000000,12.93294300,1684601999999,348112.11652606,424,7.85227300,211352.91804529,0
198,1684602000000,26918.46000000,26929.36000000,26917.68000000,26925.43000000,23.22837300,1684602899999,625398.73120790,776,16.32273100,439468.73555667,0


In [6]:
def getdata(symbol):
    frame = pd.DataFrame(client.get_historical_klines(symbol, '15m', '3000 minutes UTC'))
    frame = frame.iloc[:, 0:5]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [7]:
df = getdata(symbol)

### Indicator Calculation/ Buy Condition

In [8]:
def indicators(df):
    df['SMA_200'] = ta.trend.sma_indicator(df['Close'], window=200)
    df['stochrsi_k'] = ta.momentum.stochrsi_k(df['Close'], window=10)
    df.dropna(inplace=True)
    df['Buy'] = (df['Close'] > df['SMA_200']) & (df['stochrsi_k'] < 0.05)
    return df
    

In [9]:
indicators(df)

,Open,High,Low,Close,SMA_200,stochrsi_k,Buy
Time,,,,,,,
2023-05-20 17:15:00,26925.43,26925.45,26918.07,26924.01,26866.48855,0.106723,False


In [10]:
float(client.get_symbol_ticker(symbol=symbol)['price']) * 0.97

26116.260599999998

### Buy Limit Order: Helper Function

In [11]:
def pricecalc(symbol, limit=0.97):
    raw_price = float(client.get_symbol_ticker(symbol=symbol)['price'])
    dec_len = len(str(raw_price).split('.')[1])
    price = raw_price * limit
    return round(price, dec_len)

def quantitycalc(symbol, investment):
    info = client.get_symbol_info(symbol=symbol)
    Lotsize = float([i for i in info['filters'] if i['filterType'] == 'LOT_SIZE'][0]['minQty'])
    price = pricecalc(symbol)
    qty = round(investment/price, right_rounding(Lotsize))
    return qty

def right_rounding(Lotsize):
#     print(Lotsize)
#     splitted = str(Lotsize).split('.') 
#     if float(splitted[0]) == 1:
#         return 0
#     else:
#         return len(splitted[1])
    decimal_places = 0
    while Lotsize < 1:
        Lotsize *= 10
        decimal_places += 1
    return decimal_places

In [12]:
quantitycalc('XRPUSDT', 100)

221.4

In [13]:
right_rounding(1)

0

### Buy and Sell Function

In [14]:
def buy(investment):
    order = client.order_limit_buy(
        symbol=symbol,
        price=pricecalc(symbol),
        quantity=quantitycalc(symbol, investment)
    )
    print(order)
    pos_dict['in_position'] = True
    return order

def sell(qty):
    order = client.create_order(
        symbol=symbol,
        side='SELL',
        type='MARKET',
        quantity=qty
    )
    print(order)
    pos_dict['in_position'] = False

### Checking buying signals & Position

In [15]:
pos_dict = {'in_position': False}

In [16]:
def checkbuy(investment):
    if not pos_dict['in_position']: 
        if df.Buy.values:
            return True
    else:
        print('already in a position')

### Checking selling signals & Position

In [17]:
def checksell(order):
    order_status = client.get_order(symbol=symbol, orderId=order['orderId'])
    if pos_dict['in_position']:
        if order_status['status'] == 'NEW':
            print('Buy limit order pending')
        elif order_status['status'] == 'FILLED':
            cond1 = df.Close.values > float(order_status['price'])
            cond2 = pd.to_datetime('now') >= pd.to_datetime(order_status['updateTime'], unit='ms') + timedelta(minutes=150)
            if cond1 or cond2:
                sell(order_status['origQty'])
                print('Sell order executed')
    else:
        print('Currently not in position, no checks for selling')

### Running the bot

In [18]:
investment = 100

while True:
    df = indicators(getdata(symbol))
    
    if checkbuy(investment):
        curr_order = buy(investment)
    try:
        checksell(curr_order)
    except:
        print('Not an order yet')
    
    time.sleep(60)

Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order yet
Not an order y

c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


already in a position


c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


Not an order yet
already in a position


...

c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


Not an order yet
already in a position


c:\users\acer\appdata\local\programs\python\python38\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


Not an order yet


ReadTimeout: HTTPSConnectionPool(host='testnet.binance.vision', port=443): Read timed out. (read timeout=10)